In [ ]:
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
with open("function-calc-test/lin5qubits/scores.json",'r') as f:
    results=json.load(f)

In [ ]:
# results={}
# for i in glob('function-calc-test/lin5qubits/*/*results.json'):

#     with open(i,'r') as f:
#         results[os.path.basename(os.path.dirname(i))]=json.load(f)

In [ ]:
r2_df=pd.DataFrame.from_dict(results).T[['R2_train','R2_test']].sort_values(by='R2_test')

In [ ]:
r2_df

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,30))
ax=sns.barplot(data=r2_df.reset_index().melt(id_vars='index'),x='value',y='index',hue='variable',palette=sns.color_palette("Paired",2))
ax.tick_params(axis='x', labelrotation=90)
plt.xlabel('R$^{2}$')
plt.ylabel('ENCODER_ANSATZ')
plt.title("Linear Function Fitting with 5 Qubits")
plt.tight_layout()
plt.savefig("bar_Q5_RUD1_AL1_TS0.8_qulacs.png",dpi=300,bbox_inches="tight")

plt.show()